## Classificação Multiclasse

Usaremos a Classificação Multiclasse com Árvores de Decisão para construir um modelo capaz de prever o resultado de uma partida de futebol com 3 resutltados possíveis: vitória, derrota ou empate.

In [3]:
# Importa o findspark e inicializa
import findspark 
findspark.init()

In [4]:
# Imports
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### Carregando os Dados

In [9]:
# Criando o Spark Context
sc = SparkContext(appName = 'Classificacao Multiclasse')

23/03/20 17:16:56 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [10]:
sc.setLogLevel('ERROR')

In [12]:
# Spark Session - usada quando se trabalha com Dataframes nos Spark
spSession = SparkSession.builder.master('local').getOrCreate()

In [13]:
# Carregando os dados e geranto um RDD
dados_time_futebol = sc.textFile('dados/dataset2.csv')

In [14]:
# Colocando o RDD em cache. Esse processo otimiza a performance.
dados_time_futebol.cache()

dados/dataset2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [15]:
dados_time_futebol.count()

151

In [16]:
dados_time_futebol.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [17]:
dados_time_futebol_2 = dados_time_futebol.filter(lambda x: 'media_cartoes_recebidos' not in x)
dados_time_futebol_2.count()

150

### Limpeza e Transformação dos Dados

In [20]:
# Separando as colunas
dados_time_futebol_3 = dados_time_futebol_2.map(lambda l: l.split(','))

In [21]:
# Mapeando as colunas
dados_time_futebol_4 = dados_time_futebol_3.map(lambda p: Row(media_faltas_sofridas = float(p[0]),
                                                              media_faltas_recebidas = float(p[1]),
                                                              media_cartoes_recebidos = float(p[2]),
                                                              media_chutes_a_gol = float(p[3]),
                                                              resultado = p[4]))

In [22]:
# Converte o RDD para DataFrame do Spark
df_time = spSession.createDataFrame(dados_time_futebol_4)
df_time.cache()

DataFrame[media_faltas_sofridas: double, media_faltas_recebidas: double, media_cartoes_recebidos: double, media_chutes_a_gol: double, resultado: string]

In [23]:
df_time.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [30]:
# Criando um índice numérico para a coluna de label target
stringIndexer = StringIndexer(inputCol = 'resultado', outputCol = 'idx_resultado')

In [31]:
# Treinando o string indexer
si_model = stringIndexer.fit(df_time)

In [32]:
# Aplicando o string indexer
df_time_final = si_model.transform(df_time)

In [35]:
df_time_final.select('resultado', 'idx_resultado').distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

### Análise Exploratória de Dados

In [37]:
# Estatística descritiva
df_time.describe().show()

+-------+---------------------+----------------------+-----------------------+------------------+---------+
|summary|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|
+-------+---------------------+----------------------+-----------------------+------------------+---------+
|  count|                  150|                   150|                    150|               150|      150|
|   mean|    5.843333333333332|    3.0573333333333337|      3.758000000000001|1.1993333333333331|     null|
| stddev|   0.8280661279778625|   0.43586628493669793|     1.7652982332594667|0.7622376689603465|     null|
|    min|                  4.3|                   2.0|                    1.0|               0.1|  derrota|
|    max|                  7.9|                   4.4|                    6.9|               2.5|  vitoria|
+-------+---------------------+----------------------+-----------------------+------------------+---------+



In [38]:
# Correlação entre as variáveis
for i in df_time_final.columns:
    if not(isinstance(df_time_final.select(i).take(1)[0][0], str)) :
        print('Correlação da variável idx_resultdo com:', i, df_time_final.stat.corr('idx_resultado', i))

Correlação da variável idx_resultdo com: media_faltas_sofridas -0.460039156500237
Correlação da variável idx_resultdo com: media_faltas_recebidas 0.6183715308237437
Correlação da variável idx_resultdo com: media_cartoes_recebidos -0.6492418307641741
Correlação da variável idx_resultdo com: media_chutes_a_gol -0.5803770334306263
Correlação da variável idx_resultdo com: idx_resultado 1.0


### Pré-Processamento de Dados

In [50]:
# Criando um LabeledPoint (target, Vector[features])
# Remove colunas não relevantes para o modelo ou com baixa correlaçao
def transformaVar(row):
    obj = (row['resultado'], row['idx_resultado'], Vectors.dense([row['media_faltas_sofridas'],
                                                                  row['media_faltas_recebidas'],
                                                                  row['media_cartoes_recebidos'],
                                                                  row['media_chutes_a_gol']]))
    return obj

In [51]:
# Aplica a função
df_time_final_RDD = df_time_final.rdd.map(transformaVar)

In [52]:
df_time_final_RDD.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [53]:
 # Converte o RDD para DataFrame
df_spark = spSession.createDataFrame(df_time_final_RDD, ['resultado', 'label', 'features'])

In [54]:
df_spark.cache()

DataFrame[resultado: string, label: double, features: vector]

In [55]:
df_spark.select('resultado', 'label', 'features').show(10)

+---------+-----+-----------------+
|resultado|label|         features|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [56]:
# Dados de Treino e de Teste
(dados_treino, dados_teste) = df_spark.randomSplit([0.7, 0.3])

In [57]:
dados_treino.count()

111

In [58]:
dados_teste.count()

39

### Machine Learning

In [59]:
# Cria o objeto 
dfClassifier = DecisionTreeClassifier(maxDepth = 2, labelCol = 'label', featuresCol = 'features')

In [60]:
# Treina o objeto com dados para criar o modelo
modelo = dfClassifier.fit(dados_treino)

In [61]:
# Hiperparâmetro definido por padrão
modelo.numNodes

5

In [62]:
# Hiperparâmetro ddefinido por mim
modelo.depth

2

In [63]:
# Previsoes com dados de teste
previsoes = modelo.transform(dados_teste)

In [64]:
previsoes

DataFrame[resultado: string, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [65]:
previsoes.select('resultado', 'label', 'prediction', 'probability').collect()

[Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='derrota', label=0.0, prediction=0.0, probability=DenseVector([0.9211, 0.0789, 0.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0])),
 Row(resultado='vitoria', label=2.0, prediction=2.0, probability=DenseVector([0.0, 0.0, 1.0]))

In [67]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = 'prediction',
                                              labelCol = 'label',
                                              metricName = 'accuracy')

In [68]:
avaliador.evaluate(previsoes)

0.9487179487179487

In [69]:
# Resumindo as previsões - confusion Matrix
previsoes.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       2.0|   15|
|  0.0|       0.0|   14|
|  1.0|       1.0|    8|
|  1.0|       0.0|    2|
+-----+----------+-----+

